In [ ]:
#pip install git+https://github.com/JustAnotherArchivist/snscrape.git

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
pres = pd.read_csv('pres.csv')
pres

In [ ]:
search_phrases = {}
for c in pres['candidate']:
    phrases = [c, c.split(' ')]
    search_phrases[c] = phrases
    

In [ ]:
def get_tweets(name, year):
    tweets = []
    for i,t in enumerate(sntwitter.TwitterSearchScraper(f'{name} since:{year}-10-01 until:{year}-10-31').get_items()):
        if i>10000:
            break
        tweets.append([t.date, t.content])
    df = pd.DataFrame(tweets, columns=['date', 'tweet'])
    df['candidate'] = name
    return df

In [ ]:
tweets = get_tweets(pres['candidate'][0],pres['year'][0])
tweets

In [ ]:
tweets.to_csv('pres_tweets.csv')

In [ ]:
for i in range(1,len(pres)):
    name = pres['candidate'][i]
    year = pres['year'][i]
    df = get_tweets(name, year)
    tweets = pd.concat([tweets, df], axis=0)
    tweets.to_csv('pres_tweets.csv')
  

In [ ]:
tweets.head()

# gubenatorial candidates

In [ ]:
gub = pd.read_csv('gub.csv', index_col=0)
gub

In [ ]:
#make dictionary of candidates with state
state_dict = {gub['candidate'][i]: gub['state'][i] for i in range(len(gub))}
state_dict

In [ ]:
for x in gub['candidate']:
    if len(x.split()) <2:
        print(x)

In [ ]:
gub.iloc[65]

In [ ]:
search_phrases = {}
for i in range(len(gub)):
    c= gub['candidate'][i]
    if not c in search_phrases:
        l = c.split()[1]
        s = gub['state'][i]
        phrases = [f'governor {l}', f'{l} {s}', f'{s} {l}']
        search_phrases[c] = phrases

In [ ]:
def get_tweets2(name, year):
    tweets = []
    for phrase in search_phrases[name]:
        for i,t in enumerate(sntwitter.TwitterSearchScraper(f'{phrase} since:{year}-10-01 until:{year}-10-31').get_items()):
            if i>10000:
                break
            tweets.append([t.date, t.content])
    df = pd.DataFrame(tweets, columns=['date', 'tweet'])
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    df['candidate'] = name
    return df

In [ ]:
#testing on first candiate
g_tweets = get_tweets(gub['candidate'][0],gub['year'][0])
g_tweets

In [ ]:
g_tweets.drop_duplicates(subset=None, keep='first', inplace=True)
g_tweets = g_tweets.reset_index(drop=True)
g_tweets

In [ ]:
g_tweets
for i in range(1,len(gub)):
    name = gub['candidate'][i]
    year = gub['year'][i]
    df = get_tweets2(name, year)
    g_tweets = pd.concat([g_tweets, df], axis=0)
    g_tweets.to_csv('gub_tweets.csv')
    